In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision import models
import torchvision
from tqdm import tqdm

In [2]:
# Set random seed for reproducibility
torch.manual_seed(1)

# Define constants
NUM_CLASSES = 102  # 102 classes
BATCH_SIZE = 128
LEARNING_RATE = 0.001
NUM_EPOCHS = 5

# Data transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.4751, 0.3916, 0.3025), (0.2942, 0.2407, 0.2596))
])


# Load OxfordIIITPet dataset
train_dataset = torchvision.datasets.Flowers102(root='/shareddata', download=True,split='train',transform=transform)
test_set = torchvision.datasets.Flowers102(root='/shareddata', download=True,split='test',transform=transform)


# Create data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE,shuffle=False)

# Check if GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Define ResNet model
class ResNetModel(nn.Module):
    def __init__(self, num_classes):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet18(pretrained=True)
        in_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Linear(in_features, num_classes)

    def forward(self, x):
        return self.resnet(x)

# Create an instance of the model
model = ResNetModel(num_classes=NUM_CLASSES).to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Training loop
for epoch in tqdm(range(NUM_EPOCHS)):
    model.train()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Batch {batch_idx}/{len(train_loader)}, Loss: {loss.item():.4f}")

# Evaluation
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for data, target in tqdm(test_loader):
        data, target = data.to(device), target.to(device)
        outputs = model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

accuracy = correct / total
print(f"Test Accuracy: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1/5, Batch 0/8, Loss: 4.8313


 20%|██        | 1/5 [00:03<00:13,  3.39s/it]

Epoch 2/5, Batch 0/8, Loss: 1.3645


 40%|████      | 2/5 [00:06<00:09,  3.13s/it]

...

Epoch 4/5, Batch 0/8, Loss: 0.0790


 80%|████████  | 4/5 [00:12<00:03,  3.02s/it]

Epoch 5/5, Batch 0/8, Loss: 0.0330


100%|██████████| 49/49 [00:31<00:00,  1.55it/s]

Test Accuracy: 81.88%
